In [65]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.probability import FreqDist
import string
from wordcloud import WordCloud
from konlpy.tag import Okt
from collections import Counter
import ast

import platform
from matplotlib import font_manager, rc

plt.rcParams['axes.unicode_minus']=False

path = 'c:/Windows/Fonts/malgun.ttf'
font_name = font_manager.FontProperties(fname=path).get_name()
rc('font',family=font_name)

## good, bad 리뷰 나누고 TOP10 키워드 빈도수 세기

In [83]:
cnam = pd.read_csv('data/충남.csv')
one = pd.read_csv('data/강원.csv')
gg = pd.read_csv('data/경기.csv')
ic = pd.read_csv('data/인천.csv')
cbook = pd.read_csv('data/충북.csv')
jeju = pd.read_csv('data/제주도.csv')
seoul = pd.read_csv('data/서울.csv')
gnam = pd.read_csv('data/경남.csv')
gbook = pd.read_csv('data/경북.csv')
daegu = pd.read_csv('data/대구.csv')
daej = pd.read_csv('data/대전.csv')
busan = pd.read_csv('data/부산.csv')
sejong = pd.read_csv('data/세종.csv')
ulsan = pd.read_csv('data/울산.csv')
jnam = pd.read_csv('data/전남.csv')
jbook = pd.read_csv('data/전북.csv')
gj = pd.read_csv('data/광주.csv')

In [84]:
# 1. 긍정 / 부정 나누기
def goodbad(df):
    # 특수기호 지우기(string.punctuation = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~')
    def remove_punctuation(text):
        return ''.join([i for i in text if i not in string.punctuation])
        
    df['good'] = pd.Series(dtype=str)
    df['bad'] = pd.Series(dtype=str)
    # 모든 행을 돌리기
    for i in range(len(df)):
        # i행의 모든 평점, 리뷰
        list = eval(df.loc[i,'star&reviews'])
        go_list = []
        ba_list = []
        # 모든 리뷰 돌리기
        for hotel in range(len(list)):
            # 별점이 3점 이상 -> good
            if eval(df.loc[i,'star&reviews'])[hotel][0] >= 3.0:
                go_list.append(eval(df.loc[i,'star&reviews'])[hotel][2]) 
                
            # 별점이 3점 이하 -> bad
            elif eval(df.loc[i,'star&reviews'])[hotel][0] <= 3.0:
                ba_list.append(eval(df.loc[i,'star&reviews'])[hotel][2])
        go_list_join = ' '.join(go_list) 
        ba_list_join = ' '.join(ba_list)
        good_list_join = remove_punctuation(go_list_join)
        bad_list_join = remove_punctuation(ba_list_join)
        
        good_list_join =good_list_join.replace('\n', '')
        bad_list_join = bad_list_join.replace('\n', '')
        
        df.loc[i, 'good'] = ''.join(good_list_join)
        df.loc[i, 'bad'] = ''.join(bad_list_join)

In [4]:
# 2. 유사단어들 한 단어로 통일시키기
def replace_word_good(word, replace, df):
    for i in range(len(df)):
        re_list = []
        good_split = df.loc[i, 'good'].split()
        for j in range(len(good_split)):
            if word in good_split[j]:
                good_split[j] = replace
                re_list.append(good_split[j])
            else:
                re_list.append(good_split[j])
        df.loc[i, 'good'] = ' '.join(re_list)
    return df

In [5]:
# 2-1. 유사단어들 한 단어로 통일시키기
def replace_word_bad(word, replace, df):
    for i in range(len(df)):
        re_list = []
        good_split = df.loc[i, 'bad'].split()
        for j in range(len(good_split)):
            if word in good_split[j]:
                good_split[j] = replace
                re_list.append(good_split[j])
            else:
                re_list.append(good_split[j])
        df.loc[i, 'bad'] = ' '.join(re_list)
    return df

In [6]:
# 3. 서비스 o,x로 표시
def services(df, service_list):
    import ast
    # 새로운 열을 추가하여 'O' 또는 'X'로 표시
    for service in service_list:
        df[service] = df['services'].apply(lambda x: 'O' if service in ast.literal_eval(x) else 'X')
    return df

In [145]:
# 4. 키워드 개수 세기
def count_keyword(df, keywordlist):
    df['good_keyword_ratio'] = pd.Series(dtype=str)
    for i in range(len(df)):
        keyword_ratios = {}
        for keyword in keywordlist:
            if df.loc[i,'good'].count(keyword) != 0:
                ratio = df.loc[i,'good'].count(keyword) / len(df.loc[i,'good']) * 100
                keyword_ratios[keyword] = f'{ratio:.2f}%'
            else:
                keyword_ratios[keyword] = f'0%'
        df.loc[i,'good_keyword_ratio'] = str(keyword_ratios)

In [8]:
# 5. keyword TOPn 보여주기
def keyword_topn(df,keyword,n):
    df1 = df.sort_values(keyword,ascending=False)[['region','hotel_name','services','location','무료주차', '객실샤워실', '냉장고', '레스토랑', '엘리베이터', '카페', '짐보관가능', '피트니스',
       '미니바', '수영장', '편의점', '장애인편의시설', '사우나', '프린터사용', '라운지']][:n]
    return df1

In [24]:
# 6. 호텔마다 리뷰 토큰화
def tokens(df):
    # 단어 열 생성
    df['good_keyword'] = pd.Series(dtype=str)
    df['bad_keyword'] = pd.Series(dtype=str)
    
    for i in range(len(df)):
        good_list_join = df.loc[i,'good']
        bad_list_join = df.loc[i, 'bad']
        # 단어 토큰화
        okt = Okt()
        noun_good = []
        noun_bad = []
        for good, tag in okt.pos(good_list_join):
            if tag == 'Noun':
                noun_good.append(good)
            elif tag == 'Adjective':
                noun_good.append(good)
        for bad, tag in okt.pos(bad_list_join):
            if tag == 'Noun':
                noun_bad.append(bad)
            elif tag == 'Adjective':
                noun_bad.append(bad)

        df.loc[i,'good_keyword'] = ' '.join(noun_good)

        df.loc[i,'bad_keyword'] = ' '.join(noun_bad)

    return df

In [25]:
# 7. 호텔마다 리뷰 불용어 제거
def remove_nu(df):
    
    # 불용어 / '호텔'이란 단어도 너무 많이 나와 불용어에 추가
    stopwords_kr = pd.read_csv('data/한국어_불용어리스트.csv', encoding='CP949', sep='\t')
    stopwords = list(stopwords_kr['stopwords'])
    for i in range(len(df)):
        word_tokens_good = df.loc[i,'good_keyword'].split()
        word_tokens_bad = df.loc[i,'bad_keyword'].split()

        # 불용어 제거 / tag이용해서 'Noun'만 추출가능
        token_rv_good = [word for word in word_tokens_good if len(word) > 1 and (word not in stopwords)]
        token_rv_bad = [word for word in word_tokens_bad if len(word) > 1 and (word not in stopwords)]
    
        df.loc[i,'good_keyword'] = str(token_rv_good)
        df.loc[i,'bad_keyword'] = str(token_rv_bad)
        # good_list_join = ' '.join(token_rv_good)
        # bad_list_join = ' '.join(token_rv_bad)
    
    return df

In [165]:
# 호텔 이름에 붙어있는 불필요 정보 제거
def name(df):
    for i in range(len(df)):
        if df['hotel_name'][i].split()[0] == '[반짝특가]':
            df['hotel_name'][i] = ' '.join(df.loc[i]['hotel_name'].split()[1:])
        elif df['hotel_name'][i].split()[0] == '★당일특가★':
            df['hotel_name'][i] = ' '.join(df.loc[i]['hotel_name'].split()[1:])

In [94]:
seoul['star&reviews'][16] = "[]"

In [96]:
# 긍정 / 부정 나누고 단어 통일화하고 토큰화 후 불용어 제거
lists = [cnam,one,gg,ic,cbook,jeju,seoul,gnam,gbook,daegu,daej,busan,sejong,ulsan,jnam,jbook,gj]
for i in lists:
    goodbad(i)
    replace_word_good('깨끗','청결',i)
    replace_word_good('청결','청결',i)
    replace_word_good('깔끔','청결',i)
    replace_word_good('가성','가격',i)
    replace_word_good('가격','가격',i)
    replace_word_good('저렴','가격',i)
    replace_word_good('친절','친절',i)
    replace_word_good('사장','직원',i)
    replace_word_good('직원','직원',i)
    replace_word_bad('더러','청결',i)
    replace_word_bad('더럽','청결',i)
    replace_word_bad('비싸','가격',i)
    replace_word_bad('비쌈','가격',i)
    tokens(i)
    remove_nu(i)

In [127]:
# 지역별 긍정 키워드 TOP10
one_keywordlist = '청결 객실 시설 바다 직원 속초 이용 가격 예약 위치'.split()
cnam_keywordlist = '청결 시설 객실 이용 직원 주변 조식 가격 화장실 주차'.split()
gg_keywordlist = '청결 객실 직원 이용 시설 주변 주차 침대 조식 가격'.split()
ic_keywordlist = '청결 객실 시설 직원 이용 주변 가격 주차 체크 근처'.split()
cbook_keywordlist = '청결 객실 시설 직원 이용 주변 조식 청주 예약 생각'.split()
jeju_keywordlist = '청결 직원 가격 객실 시설 이용 침대 위치 근처 주차'.split()
seoul_keywordlist = '청결 객실 직원 이용 위치 가격 시설 체크 조식 근처'.split()
gnam_keywordlist = '청결 객실 시설 직원 이용 조식 예약 주변 바다 침대'.split()
gbook_keywordlist = '청결 객실 시설 조식 직원 경주 이용 가격 위치 가족'.split()
daegu_keywordlist = '청결 조식 시설 객실 직원 이용 주변 주차 가격 예약'.split()
daej_keywordlist = '청결 객실 가격 조식 주차 이용 직원 시설 위치 주변'.split()
busan_keywordlist = '청결 객실 직원 위치 바다 이용 가격 시설 해운대 오션'.split()
sejong_keywordlist = '청결 시설 여유 주차 식당 상태 주변 청소 침구 주차공간'.split()
ulsan_keywordlist = '청결 객실 주차 시설 위치 직원 조식 가격 침대 이용'.split()
jnam_keywordlist = '청결 객실 시설 직원 여수 조식 가격 침대 이용 바다'.split()
jbook_keywordlist = '청결 직원 객실 시설 전주 조식 마을 이용 침대 위치'.split()
gj_keywordlist = '청결 조식 시설 직원 객실 이용 주차 침대 가격 주변'.split()

In [148]:
# 지역별 긍정 키워드 TOP10을 호텔마다 적용하여 단어 비율 추출
lists = [cnam, one, gg, ic, cbook, jeju, seoul, gnam, gbook, daegu,daej, busan, sejong, ulsan, jnam, jbook, gj]

count_keyword(cnam,cnam_keywordlist)
count_keyword(one,one_keywordlist)
count_keyword(gg,gg_keywordlist)
count_keyword(ic,ic_keywordlist)
count_keyword(jeju,jeju_keywordlist)
count_keyword(gnam,gnam_keywordlist)
count_keyword(gbook,gbook_keywordlist)
count_keyword(daegu,daegu_keywordlist)
count_keyword(busan,busan_keywordlist)
count_keyword(sejong,sejong_keywordlist)
count_keyword(jnam,jnam_keywordlist)
count_keyword(jbook,jbook_keywordlist)
count_keyword(gj,gj_keywordlist)
count_keyword(cbook,cbook_keywordlist)
count_keyword(seoul,seoul_keywordlist)
count_keyword(daej,daej_keywordlist)
count_keyword(ulsan,ulsan_keywordlist)

In [ ]:
keyword_topn(gg_ic,'청결',10).head(3)

## 위도와 경도 추가하기

In [424]:
juso = pd.read_csv('df_regions_전국호텔위도경도_전체.csv')

# 위도 경도 추가시키는 함수
def jusoconcat(df,region,juso=juso):
    df1 = juso[juso['region'] == region].sort_values('location')[['위도','경도']]
    df2 = df.sort_values('location')
    # df3 = pd.concat([df2,df1],axis=1)
    df3 = pd.merge(df2, df1, on='location', how='left')
    df = df3.sort_index()
    return df

In [ ]:
# 인덱스 통일시키기
lists = [cnam, one, gg, ic, cbook, jeju, seoul, gnam, gbook, daegu,daej, busan, sejong, ulsan, jnam, jbook, gj]
city = ['충남', '강원', '경기', '인천', '충북', '제주도', '서울', '경남', '경북', '대구', '대전', '부산', '세종', '울산', '전남', '전북', '광주']

for idx, df in enumerate(lists):
    lists[idx] = df.set_index('location')

juso1 = juso.set_index('location')

# 호텔마다 위도 경도 추가하기
for idx, df in enumerate(lists):
    lists[idx] = jusoconcat(lists[idx],city[idx],juso1)
    lists[idx].drop_duplicates(subset=['hotel_name'], keep='first', inplace=True)
    lists[idx] = lists[idx].reset_index()

In [430]:
# 열 통일
new_order = ['region','hotel_name','services','location','star&reviews','good','bad','good_keyword','bad_keyword','good_keyword_ratio','위도','경도']

cnam1 = lists[0].reindex(columns=new_order)
one1 = lists[1].reindex(columns=new_order)
gg1 = lists[2].reindex(columns=new_order)
ic1 = lists[3].reindex(columns=new_order)
cbook1 = lists[4].reindex(columns=new_order)
jeju1 = lists[5].reindex(columns=new_order)
seoul1 = lists[6].reindex(columns=new_order)
gnam1 = lists[7].reindex(columns=new_order)
gbook1 = lists[8].reindex(columns=new_order)
daegu1 = lists[9].reindex(columns=new_order)
daej1 = lists[10].reindex(columns=new_order)
busan1 = lists[11].reindex(columns=new_order)
sejong1 = lists[12].reindex(columns=new_order)
ulsan1 = lists[13].reindex(columns=new_order)
jnam1 = lists[14].reindex(columns=new_order)
jbook1 = lists[15].reindex(columns=new_order)
gj1 = lists[16].reindex(columns=new_order)


In [433]:
# 개수가 달라지지 않았는지 확인
lists = [cnam1, one1, gg1, ic1, cbook1, jeju1, seoul1, gnam1, gbook1, daegu1,daej1, busan1, sejong1, ulsan1, jnam1, jbook1, gj1]
for i in lists:
    print(f'{len(i)}')


50
50
50
50
48
50
50
50
50
47
40
50
2
28
50
49
50


In [162]:
# 호텔이름에 당일특가 등 불필요한 정보 제거 
lists = [cnam, one, gg, ic, cbook, jeju, seoul, gnam, gbook, daegu,daej, busan, sejong, ulsan, jnam, jbook, gj]
for i in lists:
    name(i)

In [438]:
cnam1.to_csv('충남1.csv',index=False)
one1.to_csv('강원1.csv',index=False)
gg1.to_csv('경기1.csv',index=False)
ic1.to_csv('인천1.csv',index=False)
cbook1.to_csv('충북1.csv',index=False)
jeju1.to_csv('제주도1.csv',index=False)
seoul1.to_csv('서울1.csv',index=False)
gnam1.to_csv('경남1.csv',index=False)
gbook1.to_csv('경북1.csv',index=False)
daegu1.to_csv('대구1.csv',index=False)
daej1.to_csv('대전1.csv',index=False)
busan1.to_csv('부산1.csv',index=False)
sejong1.to_csv('세종1.csv',index=False)
ulsan1.to_csv('울산1.csv',index=False)
jnam1.to_csv('전남1.csv',index=False)
jbook1.to_csv('전북1.csv',index=False)
gj1.to_csv('광주1.csv',index=False)

------------------------------------------------------------------------------------------------------------------------------------------------

## 함수화

In [ ]:
def goodbad_wordcloud(df, region): 

    df_sr = pd.DataFrame(df[df['region']==region]['star&reviews'])
    good_list = []
    bad_list= []
    # 각 행에 대해 반복
    for i in range(len(df_sr)):
        list = eval(df_sr['star&reviews'][i])
        for hotel in range(len(list)):
                # 'star&reviews' 열의 값을 평가하고 리스트로 변환
                reviews_list = list[hotel]
                if reviews_list[0] >= 3:
                    good_list.append(reviews_list[2])
                elif  reviews_list[0] <=3:
                    bad_list.append(reviews_list[2])

    # good / bad 별 review 합치기
    good_list_join = ' '.join(good_list)
    bad_list_join = ' '.join(bad_list)
    
    # 특수기호 지우기(string.punctuation = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~')
    def remove_punctuation(text):
        return ''.join([i for i in text if i not in string.punctuation])
    good_list_join =good_list_join.replace('\n', '').replace(' ', '')
    bad_list_join = bad_list_join.replace('\n', '').replace(' ', '')
        
    good_list_join = remove_punctuation(good_list_join)
    bad_list_join = remove_punctuation(bad_list_join)
    
    # 불용어 / '호텔'이란 단어도 너무 많이 나와 불용어에 추가
    stopwords_kr = pd.read_csv('한국어 불용어 리스트.csv', encoding='CP949', sep='\t')
    stopwords_kr = list(stopwords_kr['stopwaords'])

    # 단어 토큰화
    okt = Okt()
    word_tokens_good = okt.pos(good_list_join)
    word_tokens_bad = okt.pos(bad_list_join)
    
    # 불용어 제거 / tag이용해서 'Noun'만 추출가능
    
    token_rv_good = [word for word, tag in word_tokens_good if len(word) > 1 if word not in stopwords]
    token_rv_bad = [word for word, tag in word_tokens_bad if len(word) > 1 if word not in stopwords]
    freq_good = pd.DataFrame(pd.Series(Counter(token_rv_good)).sort_values(ascending=False), columns=['freq'])
    freq_bad = pd.DataFrame(pd.Series(Counter(token_rv_bad)).sort_values(ascending=False), columns=['freq'])

    good_list_join = ' '.join(token_rv_good)
    bad_list_join = ' '.join(token_rv_bad)

    font_path = 'C:/Windows/Fonts/malgun.ttf'
    
    # 긍정 워드클라우드
    wordcloud = WordCloud(width = 600, height = 600, max_words=20000, font_path = font_path, background_color='white').generate(good_list_join)
    plt.figure(figsize=(20, 10))
    plt.imshow(wordcloud)
    plt.axis('off')
    plt.title(f"{region}의 긍정 키워드", fontsize=16)
    plt.show()
    
    # 부정 워드클라우드
    wordcloud = WordCloud(width = 600, height = 600, max_words=20000, font_path = font_path, background_color='white').generate(bad_list_join)
    plt.figure(figsize=(20, 10))
    plt.imshow(wordcloud)
    plt.axis('off')
    plt.title(f"{region}의 부정 키워드", fontsize=16)
    plt.show()
    
    return freq_good, freq_bad

In [ ]:
good, bad = goodbad_wordcloud(gg_ic,'경기')

In [ ]:
good

In [ ]:
bad

--------------------------------------------------------------------------------------------------------------------------------------------------

## 함수 분할

In [ ]:
def tokens(df,region):
    df_sr = pd.DataFrame(df[df['region']==region]['star&reviews'])
    good_list = []
    bad_list= []
    # 각 행에 대해 반복
    for i in range(len(df_sr)):
        list = eval(df_sr['star&reviews'].iloc[i])
        for hotel in range(len(list)):
                # 'star&reviews' 열의 값을 평가하고 리스트로 변환
                reviews_list = list[hotel]
                if reviews_list[0] >= 3:
                    good_list.append(reviews_list[2])
                elif  reviews_list[0] <=3:
                    bad_list.append(reviews_list[2])

    # good / bad 별 review 합치기
    good_list_join = ' '.join(good_list)
    bad_list_join = ' '.join(bad_list)
    
    # 특수기호 지우기(string.punctuation = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~')
    def remove_punctuation(text):
        return ''.join([i for i in text if i not in string.punctuation])
    good_list_join =good_list_join.replace('\n', '').replace(' ', '')
    bad_list_join = bad_list_join.replace('\n', '').replace(' ', '')
        
    good_list_join = remove_punctuation(good_list_join)
    bad_list_join = remove_punctuation(bad_list_join)
    
    # 불용어 / '호텔'이란 단어도 너무 많이 나와 불용어에 추가
    stopwords_kr = pd.read_csv('한국어 불용어 리스트.csv', encoding='CP949', sep='\t')
    stopwords_kr.loc[len(stopwords_kr)] = '호텔'

    # 단어 토큰화
    okt = Okt()
    word_tokens_good = okt.pos(good_list_join)
    word_tokens_bad = okt.pos(bad_list_join)

    return word_tokens_good, word_tokens_bad

In [ ]:
def remove_nu(word_tokens_good, word_tokens_bad):

    # 불용어 / '호텔'이란 단어도 너무 많이 나와 불용어에 추가
    stopwords_kr = pd.read_csv('data/한국어 불용어 리스트.csv', encoding='CP949', sep='\t')
    stopwords_kr.loc[len(stopwords_kr)] = '호텔'
    stopwords = stopwords_kr['휴'].tolist()
    
    # 불용어 제거 / tag이용해서 'Noun'만 추출가능
    token_rv_good = [word for word, tag in word_tokens_good if len(word) > 1 if word not in stopwords]
    token_rv_bad = [word for word, tag in word_tokens_bad if len(word) > 1 if word not in stopwords]
    freq_good = pd.DataFrame(pd.Series(Counter(token_rv_good)).sort_values(ascending=False), columns=['freq'])
    freq_bad = pd.DataFrame(pd.Series(Counter(token_rv_bad)).sort_values(ascending=False), columns=['freq'])

    good_list_join = ' '.join(token_rv_good)
    bad_list_join = ' '.join(token_rv_bad)

    return good_list_join, bad_list_join, freq_good, freq_bad

In [ ]:
def word_colud(good_list_join,bad_list_join):
    font_path = 'C:/Windows/Fonts/malgun.ttf'
    
    # 긍정 워드클라우드
    wordcloud = WordCloud(width = 600, height = 600, max_words=20000, font_path = font_path, background_color='white').generate(good_list_join)
    plt.figure(figsize=(20, 10))
    plt.imshow(wordcloud)
    plt.axis('off')
    plt.title(f"{region}의 긍정 키워드", fontsize=16)
    plt.show()
    
    # 부정 워드클라우드
    wordcloud = WordCloud(width = 600, height = 600, max_words=20000, font_path = font_path, background_color='white').generate(bad_list_join)
    plt.figure(figsize=(20, 10))
    plt.imshow(wordcloud)
    plt.axis('off')
    plt.title(f"{region}의 부정 키워드", fontsize=16)
    plt.show()

## TEST

In [ ]:
gg_ic = pd.read_csv('df_gg_ic.csv')

In [ ]:
good, bad = tokens(gg_ic,'충남')

In [ ]:
bad_adj =[]
for word, tag in bad:
    if tag == 'Adjective':
        bad_adj.append(word)
v = ''.join(bad_adj)
v

In [ ]:
noun = []
for word, tag in bad:
    if tag == 'Noun':
        noun.append((word, tag))
    elif tag == 'Adjective':
        if word in ['불친절','시끄','불편','눅눅','간지럽','지저분','더러워','건조','청결','좁','낡','오래']:
            noun.append((word, tag))
noun

In [ ]:
a,b,c,d = remove_nu(good,noun)

In [ ]:
a_list = a.split()

In [ ]:
df_a = pd.DataFrame(a_list)

In [ ]:
df_a.to_csv('a')

In [ ]:
df_a.values.tolist()[:10]

In [ ]:
words = []
for i in df_a.values.tolist():
    word = i[0]
    words.append(word)
    text = ' '.join(words)

text
# range(len(df_a.values.tolist()))

### 같은 의미 단어 동일화

In [ ]:
# 'aa'라는 단어가 들어간 문자열 찾아서 단어 통일 시키기

def replace_word(word, replace, text_list):
    for i in range(len(text_list)):
        if word in text_list[i]:
            text_list[i] = replace
    return text_list

In [ ]:
a_list

In [ ]:
a_list = replace_word('깨끗','청결', a_list)
a_list = replace_word('깔끔','청결', a_list)
a_list = replace_word('가성','가격', a_list)
a_list = replace_word('가격','가격', a_list)
a_list = replace_word('좋','만족', a_list)
a_list = replace_word('괜찮','만족', a_list)
a_list = replace_word('훌륭','만족', a_list)

## 결과

In [ ]:
# 긍정 워드클라우드
font_path = 'C:/Windows/Fonts/malgun.ttf'
wordcloud = WordCloud(width = 600, height = 600, max_words=50, font_path = font_path, background_color='white').generate(text)
plt.figure(figsize=(10, 10))
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

In [ ]:
# okt pos 확인하기(word_tokens_good, word_tokens_bad)
tokens(df,region)

# 불용어 처리 후 워드클라우드에 넣을 text문장으로 join (good_list_join, bad_list_join, freq_good, freq_bad)
remove_nu(word_tokens_good, word_tokens_bad)

# 워드 클라우드 하기
word_colud(good_list_join,bad_list_join)